<a href="https://colab.research.google.com/github/TottiPuc/Machine_learning/blob/master/HugginFaceClasificacionPrecios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Definiendo el problema del negocio

* el primer paso en un projecto de ML es tener un panorama claro del problema del negocio que estasmo tratando de resolver
* en este ejemplo vamos a asumir que trabajamos vendiendo ropa, y que tenemos millones de diferentes productos, y tratamos de entender los comentarios y la satisfaccion de nuesrso clientes, recolectado informacion a lo largo de internet de nuestro producto y servicio y catalgoandola ella
* existen muchas formas de rankear las revisiones 
  * analisis de sentimiento
  * deteccion de emociones
  * dato numerico

en este caso vamos con las revisiones de estrellas que dejan los clientes en las compras de amazone

En este sentido el problema seria 

"dada una opinion en ingles sobre zapatos, quiero predecir un rankin de estrellas entre 1 y 5 para entender que tan satisfecho el cliente esta con su compra "

# Enmarcando el problema de negocio como un problema de machine learning
para esto necesitamos entrenar un modelo de clasificacion multiclass, el modelo debe ser capaz de predecir la probabilidad de cada una de las 5 estrellas aparecer

Necesitamos tambien definir una metrica que nos diga que tan bien el modelo esta prediciendo, para clasificaion multiclasese puede iniciar con metricas con accuracy o f1-score

asi que la primera definicion para nuestro modelo es "dado un conjunto de test de revisiones en ingles referentes a zapatos, sedesea clasificar cada revision deacurdo al rankin de estrellas entre 1 y 5, con un accuracy de almenos xx% y un f1-score de almenos yy%"



# Escogiendo el modelo
uno de los beneficion con las arquitecturas transformers es que son bastante buenos para realizar trasnfer learning. asi que en este caso lo que iniciaremos es con ir a huginface hub y explorar english language text clasification models 

In [ ]:
!pip -q install huggingface_hub --upgrade

     |████████████████████████████████| 163 kB 7.9 MB/s 


In [ ]:
!pip -q install datasets huggingface_hub --upgrade

     |████████████████████████████████| 431 kB 9.0 MB/s 
     |████████████████████████████████| 115 kB 68.4 MB/s 
     |████████████████████████████████| 212 kB 63.8 MB/s 
     |████████████████████████████████| 127 kB 41.1 MB/s 


In [ ]:
import datasets
print(datasets.__version__)

2.5.1


#Cargando y explorando

In [ ]:
from datasets import load_dataset
dataset = load_dataset('amazon_us_reviews','Shoes_v1_00', split = 'train')
print('{:.2f} GB'.format(dataset.size_in_bytes/1024/1024/1024))
print(dataset.shape)

Generating train split:   0%|          | 0/4366916 [00:00<?, ? examples/s]

Dataset amazon_us_reviews downloaded and prepared to /root/.cache/huggingface/datasets/amazon_us_reviews/Shoes_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563. Subsequent calls will reuse this data.
2.26 GB
(4366916, 15)


In [ ]:
dataset = load_dataset('amazon_us_reviews','Shoes_v1_00', split = 'train[:10%]')
print('{:.2f} GB'.format(dataset.size_in_bytes/1024/1024/1024))
print(dataset.shape)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


2.26 GB
(436692, 15)


In [ ]:
print(dataset.column_names)

['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']


In [ ]:
dataset[0]

{'marketplace': 'US',
 'customer_id': '18069663',
 'review_id': 'R3P2HIOQCIN5ZU',
 'product_id': 'B000XB31C0',
 'product_parent': '265024781',
 'product_title': "Minnetonka Men's Double Deerskin Softsole Moccasin",
 'product_category': 'Shoes',
 'star_rating': 1,
 'helpful_votes': 0,
 'total_votes': 0,
 'vine': 0,
 'verified_purchase': 1,
 'review_headline': '.',
 'review_body': "Do not buy: really didn't start to wear them until May of 2016. Junk, they are falling apart. The outer sole is so thin that although I wear them almost completely in the house on rugs the inner padding is showing through in the heel. My previous pair from the same company lasted 5 years before I threw them out.  I'm sorry I didn't wear them more often when I first got them as I would have returned them immediately",
 'review_date': '2015-08-31'}

Hay muchas columnas que no son necesarias, solo nos vamosa quedar con las que realmente nos interesan

In [ ]:
dataset=dataset.remove_columns(['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_date'])
dataset[0]

{'star_rating': 1,
 'review_body': "Do not buy: really didn't start to wear them until May of 2016. Junk, they are falling apart. The outer sole is so thin that although I wear them almost completely in the house on rugs the inner padding is showing through in the heel. My previous pair from the same company lasted 5 years before I threw them out.  I'm sorry I didn't wear them more often when I first got them as I would have returned them immediately"}

vamos a analisar que no tengamos ningun valor inesperado en el ranking de estrellas

In [ ]:
dataset.unique('star_rating')

[1, 5, 4, 3, 2]

Ahora vamos a mirar la distribucion que tiene el rankin de esterllas, esto nos mostrara que tan facil podemos convertir un datast a pandas con el fin de poder usar funciones conocidas y podremos observar que tan bien balanceado esta.

In [ ]:
dataset.to_pandas().value_counts('star_rating')

star_rating
5    265295
4     84165
3     39765
1     24445
2     23022
dtype: int64

como podemos observar el dataset es realmente desbalanceado en cuanto a ratings, por eso vamos a rebalancear el dataset y obtener soo 20000 registros de cada clase

In [ ]:
import pandas as pd
dataset_pd = dataset.to_pandas()
dataset_pd_balanced = pd.DataFrame(columns=dataset.column_names)
for stars in range(1,6):
  data = dataset_pd[dataset_pd['star_rating']==stars][:20000]
  dataset_pd_balanced = pd.concat([dataset_pd_balanced,data]) 

Ahora podemos cambiar al formato que requiere los modelos de huggin face

In [ ]:
dataset_balanced = datasets.Dataset.from_pandas(dataset_pd_balanced, preserve_index=False) # dataset en formato huggingface
print(dataset_balanced)

Dataset({
    features: ['star_rating', 'review_body'],
    num_rows: 100000
})


In [ ]:
dataset_pd_balanced  # dataset en formato pandas

,star_rating,review_body
0,1,Do not buy: really didn't start to wear them u...
19,1,Tissue paper is this thicker than this. It ri...
33,1,Not what I expected would not order again
39,1,These boots were order as the 9c which is the ...
44,1,"Cheep, made in China although they advertised ..."
...,...,...
32783,5,excelente
32789,5,Nice quality. Love the choices
32790,5,They are adorable and now my two boys have sho...
32791,5,"Looks good, wears good, and price was good."


In [ ]:
dataset_pd_balanced.value_counts('star_rating')

star_rating
1    20000
2    20000
3    20000
4    20000
5    20000
dtype: int64

una cosa a tener en cuenta aqui es que las star inician en 1, pero para los transformers loas etiquetas deben inicar en cero, asi que vamos a disminuir en unos todas las estrellas con la funcion map

In [ ]:
def disminuir_stars(row):
  return {
      "star_rating" : row['star_rating'] -1 
  }

In [ ]:
dataset_balanced = dataset_balanced.map(disminuir_stars)

  0%|          | 0/100000 [00:00<?, ?ex/s]

Ahora renombremos las columnas a lo que el modelo requiere

In [ ]:
dataset_balanced = dataset_balanced.rename_column('star_rating','labels')
dataset_balanced = dataset_balanced.rename_column('review_body','text')

In [ ]:
dataset_balanced[0]

{'labels': 0,
 'text': "Do not buy: really didn't start to wear them until May of 2016. Junk, they are falling apart. The outer sole is so thin that although I wear them almost completely in the house on rugs the inner padding is showing through in the heel. My previous pair from the same company lasted 5 years before I threw them out.  I'm sorry I didn't wear them more often when I first got them as I would have returned them immediately"}

Ahora que ya tenemos listos los datos vamos a dividir los datos en entrenamiento y test

In [ ]:
dataset_split = dataset_balanced.train_test_split(test_size=0.1, shuffle=True, seed=59)

In [ ]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 10000
    })
})

##Ahora almacenamos el dataset y lo subimos al hub

para realizar este almacaenado del dataset en el formato de hugging face que es Apache Arrow

In [ ]:
dataset_split.save_to_disk('data/')

Flattening the indices:   0%|          | 0/90 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
dataset_split.push_to_hub(repo_id='test_amazon')

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Finalmente guardamos el dataset tambien en fomrato csv

In [ ]:
import os
os.makedirs('data_csv', exist_ok=True)
dataset_split['train'].to_csv('data_csv/amazon_shoe_reviews_train.csv', header=True, index=False)
dataset_split['test'].to_csv('data_csv/amazon_shoe_reviews_test.csv', header=True, index=False)

Creating CSV from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1828484

#Ahora vamos a entrenar el modelo

In [ ]:
!git lfs install

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.


In [ ]:
!pip -q install torch scipy transformers datasets widgetsnbextension ipywidgets huggingface_hub --upgrade

     |████████████████████████████████| 4.9 MB 5.1 MB/s 
     |████████████████████████████████| 431 kB 40.0 MB/s 
     |████████████████████████████████| 2.0 MB 65.3 MB/s 
     |████████████████████████████████| 134 kB 51.5 MB/s 
     |████████████████████████████████| 163 kB 62.1 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
     |████████████████████████████████| 212 kB 47.3 MB/s 
     |████████████████████████████████| 115 kB 58.4 MB/s 
     |████████████████████████████████| 127 kB 58.1 MB/s 
     |████████████████████████████████| 1.6 MB 55.8 MB/s 


In [ ]:
import transformers
import datasets

In [ ]:
print(transformers.__version__)
print(datasets.__version__)

4.22.2
2.5.1


In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_dataset, load_from_disk

primero definimos el modelo base del cual vamos a partir asi como algunos hiperparametros

In [ ]:
base_model_id = 'distilbert-base-uncased'

epochs = 1
num_labels = 5
learning_rate = 5e-5
train_batch_size = 12 
eval_batch_size = 16
save_strategy = 'no'
save_steps = 1000

output_data_dir = './output'
model_dir = './model'

In [ ]:
#cargar el modelo opcion 1 de hugginface hub
dataset_name = 'ChristianArcos/test_amazon'
dataset = load_dataset(dataset_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 90000
    })
})

In [ ]:
#cargar el modelo opcion 2 del disco 
dataset = load_from_disk('./data')

FileNotFoundError: ignored

In [ ]:
dataset['train'][0]

{'labels': 3,
 'text': 'Good shoe for office work. They will scuff very easy so be aware.'}

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

Ahora definimos todas las metricas en una funcion las cuales queremos que aparezcan en la etapa de entrenamiento

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall , f1, _ = precision_recall_fscore_support(labels,preds)
  acc = accuracy_score(labels, preds)
  return {'accuracy' : acc, 'f1' : f1, 'precision' : precision, 'recall' : recall}


Ahora descargamos el modelo y su tokenizador del huggin face hub

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels = num_labels)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

Ahora creamos la funcion para tokenizar el dataset, lo procesaremos en batches para agilizar y aplicaremos esta funcion a los dos datasets con map() funcion diponible en dataset library

In [ ]:
def tokenize(batch):
  return tokenizer(batch['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train_dataset

Dataset({
    features: ['labels', 'text', 'input_ids', 'attention_mask'],
    num_rows: 90000
})

Ahora removemos la columna 'text' que ya no se requiere mas

In [ ]:
train_dataset = train_dataset.remove_columns(['text'])
test_dataset = test_dataset.remove_columns(['text'])

In [ ]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 90000
})

Ahora definimos los TrainingArguments para nuesttro trabajo de entrenamiento: (hyperparamettros donde almacenar los modelos et) 

In [ ]:
from transformers.trainer_utils import EvaluationStrategy
hub_model_id = 'ChristianArcos/distilbert-amazon-shoe-reviews-test'

training_args = TrainingArguments(
    hub_model_id = hub_model_id,   # esto es donde pondremos el modelo despues de entrenar
    output_dir = model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size = eval_batch_size,
    save_strategy= save_strategy,
    save_steps=save_steps,
    evaluation_strategy='epoch',
    learning_rate=learning_rate,
)

ahora usamos el objeto de entrenamietno trainer para poner todo junto

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

ahora estamos listos para entrenar

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 90000
  Num Epochs = 1
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 7500
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

#Almacenamos el modelo y hacemo push a hugginface hub

In [ ]:
trainer.save_model(model_dir+'_local')
trainer.push_to_hub()

#Tambien podemos predecir localmente con el pipeline de transformers

In [ ]:
from trasnformers import pipeline

#opcion1 cargar el modelo de hugging face hub
clasificador = pipeline('text-classification', model=hub_model_id)

#opcion2 cargar el modelo de archivos locales
clasificador = pipeline('text-classification', model='./model')

Ahora hacemos laspredicciones

In [ ]:
clasificador('the shoes fell to pieces after a few days ')

In [ ]:
clasificador('this is a very confortable pair of shoes')

se puede hacer deploy del modelo en huggingface gratio o stremlest y hacer muchos mas visible para personals externo el video al enlace se encuentra a continuacion [video](https://www.youtube.com/watch?v=1v4Ut7umOUw&ab_channel=FourthBrainAI) 